<a href="https://colab.research.google.com/github/davidemichelon11/NLU/blob/main/NLU_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('punkt')
nltk.download('movie_reviews')
nltk.download('vader_lexicon')

mr = movie_reviews
neg = mr.paras(categories='neg')
pos = mr.paras(categories='pos')

**BASELINE via SVM**

In [ ]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

In [ ]:
# Considering the constant TRAINING_SPLIT, the function spits between training and test (data and labels)
def split_samples(cor, val, train_samples, test_samples, train_labels, test_labels):
  training_limit = int(len(cor)*TRAINING_SPLIT)
  for i, value in enumerate(cor):
    if i < training_limit:
      train_samples.append(value)
      train_labels.append(val)
    else:
      test_samples.append(value)
      test_labels.append(val)

In [ ]:
from sklearn.svm import SVC
import random
def compute_SVM(pos, neg):
  vectorizer = CountVectorizer()
  classifier2 = SVC()
  TRAINING_SPLIT = 0.80
    
  train_samples = []
  train_labels = []
  test_samples = []
  test_labels = []

  split_samples([str(d) for d in neg], 0, train_samples, test_samples, train_labels, test_labels)
  split_samples([str(d) for d in pos], 1, train_samples, test_samples, train_labels, test_labels)

  vectors = vectorizer.fit_transform(train_samples + test_samples)
  classifier2.fit(vectors[:len(train_samples)], train_labels)
  labels_pred = classifier2.predict(vectors[len(train_labels):])
  print(classification_report(test_labels, labels_pred, digits=3))

In [ ]:
def doc2string(doc):
  return " ".join([w for sent in doc for w in sent])

**VADER Baseline**

In [ ]:
# Analyse complete review
from nltk.sentiment.vader import SentimentIntensityAnalyzer, VaderConstants

analyzer = SentimentIntensityAnalyzer()
labels = numpy.array([0] * len(neg) + [1] * len(pos))
prediction_val = [analyzer.polarity_scores(doc2string(v)) for v in (pos + neg)]
prediction_labels = [0 if p['pos'] > p['neg'] else 1 for p in prediction_val]

print(classification_report(labels, prediction_labels, digits=3))

In [ ]:
# Analyse each sentence of review, sum sentences contribution as 1
prediction_labels = []

for rev in (pos+neg):
  pos_ = 0
  neg_ = 0
  for sent in rev:
    p = analyzer.polarity_scores(" ".join([w for w in sent]))
    if p['pos'] > p['neg']: pos_ += 1
    else: neg_ += 1
  prediction_labels.append(0 if pos_ > neg_ else 1)
print(classification_report(labels, prediction_labels, digits=3))

# Analyse each sentence of review, sum sentences contribution as 1
prediction_labels = []

for rev in (pos+neg):
  pos_ = 0
  neg_ = 0
  for sent in rev:
    p = analyzer.polarity_scores(" ".join([w for w in sent]))
    if p['pos'] > p['neg']: pos_ += p['pos']
    else: neg_ += p['neg']
  prediction_labels.append(0 if pos_ > neg_ else 1)
print(classification_report(labels, prediction_labels, digits=3))